In [1]:
import pandas as pd
import ast
import re

from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

import plotly.graph_objects as go

The dessi dataset can be downloaded from   
https://www.kaggle.com/datasets/sensitivedetection/dessi-dataset-for-structured-sensitive-information  
where the only the DeSSI_v2 folder is necessary

# Evaluation - Presidio Confusion Matrix on Dessi Data

Use a Presidio Confusion Matrx to find label errors for Dessi Data  
Use all Predictions of Presidio without any thresholds so that no wrong predictions are ignored

In [2]:
dessi_train_labels = pd.read_csv("DeSSI_v2/train_labels.csv")
dessi_train = pd.read_csv("DeSSI_v2/train.csv")

dessi_dev_labels = pd.read_csv("DeSSI_v2/dev_labels.csv")
dessi_dev = pd.read_csv("DeSSI_v2/dev.csv")

dessi_test_labels = pd.read_csv("DeSSI_v2/test_labels.csv")
dessi_test = pd.read_csv("DeSSI_v2/test.csv")

dessi_all = pd.concat([dessi_train, dessi_dev, dessi_test], axis=1)
dataset_split = ["train"] * dessi_train.shape[1] + ["validation"] * dessi_dev.shape[1] + ["test"] * dessi_test.shape[1]
dataset_split_df = pd.DataFrame(dataset_split).T
dataset_split_df.columns = dessi_all.columns
dessi_all = pd.concat([dessi_all, dataset_split_df]).reset_index(drop=True)

dessi_all_labels = pd.concat([dessi_train_labels, dessi_dev_labels, dessi_test_labels], axis=0).reset_index(drop=True)
dataset_split_df = pd.DataFrame(dataset_split)
dataset_split_df.columns = ["split_type"]
dessi_all_labels = pd.concat([dessi_all_labels, dataset_split_df], axis=1).reset_index(drop=True)

C:\Users\Luca\AppData\Local\Temp\ipykernel_15780\3396044622.py:2: DtypeWarning: Columns (62,107,161,241,255,324,449,503,686,720,807,812,889,934,1073,1113,1133,1154,1158,1242,1317,1333,1394,1408,1470,1553,1577,1674,1729,1762,1872,1913,1940,2045,2131,2139,2209,2288,2404,2436,2601,2627,2663,2779,2841,2960,2994,3061,3063,3080,3129,3136,3250,3319,3439,3464,3499,3520,3596,3637,3649,3748,3778,3787,3808,3902,3907,3917,3951,4009,4034,4037,4069,4129,4210,4218,4252,4325,4359,4386,4479,4646,4649,4652,4710,4936,4939,5013,5065,5069,5348,5352,5356,5399,5476,5702,5736,5889,5962,6072,6094,6107,6140,6159,6337,6360,6383,6407,6415,6423,6507,6555,6567,6590,6593,6689,6809,6815,6920,6995,7042,7127,7163,7229,7295,7301,7319,7321,7354,7399,7403,7465,7474,7593,7594,7694,7719,7798,7872,7979,8179,8298,8308,8473,8542,8565,8568,8570,8579,8633,8662,8707,8835,8846,8859,8899,8933,9024,9084,9126,9170,9184,9198,9251,9329,9412,9505,9588,9592,9610,9652,9803,9818,9907,9908,9952,9975,10073,10099,10136,10161,10359,10369,10407

In [3]:
def read_files(method):
    train = pd.read_csv(f"../../Presidio/old_predictions/dessi_results/train/results_{method}.csv")
    val = pd.read_csv(f"../../Presidio/old_predictions/dessi_results/val/results_{method}.csv")
    test = pd.read_csv(f"../../Presidio/old_predictions/dessi_results/test/results_{method}.csv")
    return train, val, test

def concat_results(train, val, test):
    all_results = pd.concat([train, val, test], axis=1)
    dataset_split = ["train"] * dessi_train.shape[1] + ["validation"] * dessi_dev.shape[1] + ["test"] * dessi_test.shape[1]
    dataset_split_df = pd.DataFrame(dataset_split).T
    dataset_split_df.columns = all_results.columns
    return pd.concat([all_results, dataset_split_df]).reset_index(drop=True).T.reset_index(drop=True).T     #reset index and column names

results_method3_train, results_method3_val, results_method3_test = read_files("columnwise")

results_method_3_all = concat_results(results_method3_train, results_method3_val, results_method3_test)

The Classes of Presidio must be mapped to the classes of dessi to compare both

In [4]:
# Mapping dictionary
# IN_VEHICLE_REGISTRATION, IP_ADDRESS, MEDICAL_LICENSE, URL, US_BANK_NUMBER are matched to "Other_data"
CATEGORY_MAP = {
    "AU_TFN": "NIN",
    "AU_MEDICARE": "NIN",
    "IN_AADHAAR": "NIN",
    "IN_PAN": "NIN",
    "IN_PASSPORT": "Passport",
    "UK_NHS": "NIN",
    "US_ITIN": "NIN",
    "US_SSN": "NIN",
    "AU_ABN": "Organization",
    "AU_ACN": "Organization",
    "CREDIT_CARD": "CCN",
    "DATE_TIME": "Date",
    "shared DATE_TIME": "Date",
    "EMAIL_ADDRESS": "Email",
    "IBAN_CODE": "IBAN",
    "LOCATION": ["Geolocation", "Address", "GPE"],
    "shared LOCATION": ["Geolocation", "Address", "GPE"],
    "NRP": ["Nationality", "Religion"],  
    "shared NRP": ["Nationality", "Religion"],
    "PERSON": "Person",
    "shared PERSON": "Person",
    "PHONE_NUMBER": "Phone_number",
    "shared PHONE_NUMBER": "Phone_number",
    "US_DRIVER_LICENSE": "ID_Card",
    "shared US_DRIVER_LICENSE": "ID_Card",
    "US_PASSPORT": "Passport",
    "shared IN_PAN": "NIN",
}

In [5]:
def get_categories(strings):
    if strings == []:
        return ["Other_data"]
    results = []
    for s in strings:
        category = CATEGORY_MAP.get(s, "Other_data")
        if isinstance(category, list):
            results.extend(category) 
        else:
            results.append(category)  
    return results

def convert_labels(label_str):
    return [label.strip() for label in label_str.replace('{', '').replace('}', '').replace("'", "").split(',')]

def build_dataframe(results_df, threshold_score, threshold_count):
    height, width = results_df.shape
    results_copy = results_df.copy()
    print(f"Threshold score: {threshold_score}\nThreshold count: {threshold_count}")
    empty_row1 = [[] for _ in range(width)]
    empty_row2 = [[] for _ in range(width)]
    empty_row3 = [[] for _ in range(width)]
    results_copy.loc[height] = empty_row1.copy()
    results_copy.loc[height+1] = empty_row2.copy()
    results_copy.loc[height+2] = empty_row3.copy()
    for i in range(width):
        series = pd.Series([item for sublist in results_copy.iloc[2:height:2, i]
                        for item in ast.literal_eval(sublist)], dtype="object").value_counts()
        for a, b in series.items():
            if b > threshold_count:
                results_copy.iloc[height, i].append(a)
                results_copy.iloc[height+1, i].append(b)
    for i in range(width):
        entities = [re.sub(r'_\d+(\.\d+)?$', '', item)
                    for item in results_copy.iloc[height, i] if float(re.search(r'(\d+\.\d+)', item).group(1)) >= threshold_score]
        mapped_entities = get_categories(entities)
        results_copy.iloc[height+2, i] = str(set(mapped_entities))

    results_copy.loc[0] = results_copy.loc[0].apply(convert_labels)
    results_copy.loc[height+2] = results_copy.loc[height+2].apply(convert_labels)
    return results_copy

def compute_performance(results, confusion = False):
    results_copy = results.copy()
    mlb = MultiLabelBinarizer()
    y_true = mlb.fit_transform(results_copy.loc[0])
    y_pred = mlb.transform(results_copy.loc[results_copy.shape[0]-1])
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None, zero_division=0)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    evaluation = pd.DataFrame({
        'Class': mlb.classes_,
        'P': precision,
        'R': recall,
        'F1': f1,
        'S': support
    }).sort_values("S", ascending=False).reset_index(drop=True)
    print(f'Micro F1 Score: {f1_micro:.4f}')
    print(f'Macro F1 Score: {f1_macro:.4f}')
    print(f'Weighted F1 Score: {f1_weighted:.4f}')
    
    if confusion == True:
        y_true_df = pd.DataFrame(y_true)
        y_pred_df = pd.DataFrame(y_pred)

        confusion_matrix = []
        for i in range(20):
            pred_true_compare = y_pred_df.loc[y_pred_df.index.isin(y_true_df.loc[y_true_df[i] == 1].index)]
            arr_label = []
            for j in range(20):
                val = pred_true_compare.loc[:, j].value_counts(normalize=True).get(1)
                arr_label.append(val.item() if val != None else 0)
            confusion_matrix.append(arr_label)
        fig = go.Figure(go.Heatmap(z=confusion_matrix, x=mlb.classes_, y=mlb.classes_, colorscale='Blues'))
        fig.update_layout(title_text='Confusion Matrix', height=800, width=800)
        fig.update_xaxes(title="Predicted Label")
        fig.update_yaxes(title="True Label")
        fig.show()
    return evaluation

In [6]:
results_method3_thresholds = build_dataframe(results_method_3_all, 0, 0)
evaluation_method3 = compute_performance(results_method3_thresholds, confusion = True) # Best threshold_score and threshold_count

print(evaluation_method3)

Threshold score: 0
Threshold count: 0
Micro F1 Score: 0.2469
Macro F1 Score: 0.2354
Weighted F1 Score: 0.3321


           Class         P         R        F1     S
0     Other_data  0.137213  0.242951  0.175377  6561
1   Phone_number  0.377567  0.990493  0.546727  4418
2        Address  0.169818  0.843240  0.282704  3802
3         Person  0.204139  0.999721  0.339046  3582
4            NIN  0.219195  0.651773  0.328062  3469
5          Email  1.000000  0.995926  0.997959  3191
6           Date  0.153270  0.985622  0.265286  2782
7            GPE  0.104825  0.911142  0.188020  2172
8   Organization  0.001077  0.001445  0.001234  2076
9    Geolocation  0.035701  0.327502  0.064384  2058
10     SWIFT/BIC  0.000000  0.000000  0.000000   240
11          IBAN  0.892019  0.892019  0.892019   213
12           CCN  0.345098  0.807339  0.483516   109
13      Passport  0.046512  0.388889  0.083086   108
14        Gender  0.000000  0.000000  0.000000    94
15      Religion  0.012819  1.000000  0.025313    93
16     Sexuality  0.000000  0.000000  0.000000    92
17   Nationality  0.012543  1.000000  0.024775

In [7]:
dessi_labels_cleaned =pd.read_csv("dessi_cleaned/all_labels_cleaned.csv")
wrong_columns = list(pd.read_csv("dessi_cleaned/wrong_columns.csv")["0"])

# Analyze Other Data

Look out which personal-related classes have been predicted for other data:  
Most frequent classes predicted instead of other data:  
Address, Date, GPE, Geolocation, ID_Card, Nationality, Person, Phonenumber, Religion

In [8]:
def contains_other_data(lst):
    return ["Other_data"] == lst
def not_contains_other_data(lst):
    return ["Other_data"] != lst
cond_first_row = results_method3_thresholds.iloc[0].apply(contains_other_data)
cond_last_row = results_method3_thresholds.iloc[-1].apply(not_contains_other_data)
df_other = results_method3_thresholds[results_method3_thresholds.columns[cond_first_row & cond_last_row]]
df_other.shape

(205, 5587)

In [9]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Address" in x)]].columns
pd.set_option('display.max_columns', 250)
dessi_all.iloc[:,[int(a) for a in indi]].head()

,kozakymqwsyw,pwd_argon,zudlvyipnzyv,passwordquestion,passwordanswer,customer_iban.1,dytdfmxwosdh,ddugstbrmunq,secret,acecsuuiyvic,housing_benefits_details,user_ip_address_v4,userid.3,store_id.1,iwkoypulwygs,pst_cnpt_tp_id,staff_id.1,manager_id,nxiomwqhiekk,ip_address,ipv6,bankcin,iebepvwfjyva,swiftno,internet_6,employeesid.1,nhjxxbhhurfg,password,swiftno.1,number_called_type_sid.1,user_identifier,managersid.2,xeamifuwcjeu,ffzrcnqivnto,internet_6.1,product_category_level_description,fsiqnkhkokrp,nnboipmvpshi,ip,pwd_argon.2,riuqpfjjbfav,passwordanswer.1,swiftno.2,orderid.2,sha512,nml_rate_prd_id,internet_4,mfgzbaqjrsfq,sex_id,department_id.2,userid.5,employee_id.7,svobwcxurtxo,aolmoyunoldz,pwd_argon.3,dtxgjqhaevni,user_ip_address_v6,wjrbmxmyrxll,name.6,page_details,userid.6,password_hash,exchange_cur,personid.3,dateupdated.4,lkudiyucwlem,program_description,passwordquestion.1,ip_address.1,ein,fgpnvizunbso,username,ipv6.1,customer_id.8,status_description,gkuihfafiwxr,gsxmyhrgohzi,secret.1,customer_iban.3,bcayvbfyibud,user_identifier.1,gmfxjjsywfij,status_description.1,password.1,secret.2,usr_name,qsvkbasevita,tyzuretyfemi,hpfhlljssqvj,parking_details,swiftno.3,ldrwwaganthi,mrsxyvkoapqy,ip-6,usr_name.1,xejmffsuhigx,userid.11,lupdt_ppn_id,iprwdpglsxzk,byhygouqyani,xwqbpkpturay,user,managersid.7,lupdt_ppn_id.1,vtwhyrldrnfj,zbyeytlqdddu,ip_6_address,ip4,ofaewcwxlmzu,uwbrarliwgsw,name.14,accountnum,iban.2,salt.1,name.15,userid.13,opaxvgjoxpkl,cbkezhhsulbv,program_description.1,prsykhhvpwnk,pwd_hash,user_key,nhclxdaahywb,internet_4.1,user_ip_address_v4.1,...,salary.15,departmentsid.8,sha512.2,accountid.16,pwd_hash.1,ip4.2,ip.6,material_id.1,employeesid.11,pwd_argon.2,eubcpmzprtjy,mlmyhnhdhddt,sex.2,married.27,salt.2,userid.11,department_id.7,chosen_user_name.1,orderid.7,uxvflqodedco,personid.11,gender.2,accountnum,department_id.8,passwordquestion.6,name.25,banksuffix.3,tdllqwflabab,userid.12,uatjmpclawyc,passwordquestion.7,ujmbwigbqtjc,cmjwhjuaacrv,rbujkwbfpgze,customer_bank_account.3,dkcvmigfazvn,hkpzfknxomvd,manager_id.11,traffic_sid.9,order_status_description.1,adquntsycyxf,user_identifier.1,lxtsuqbksfsx,itbxdfeprimq,traffic_sid.10,store_id.11,int_rate_inl_prd_of_fix_id.1,internet_6.5,staff_id.13,aivuliomguuz,customer_details,pstexpxsnjdj,passwordquestion.8,cmwsckgvbbeh,ip6.5,edmwgugfrwfu,internet_6.6,sha512.3,sllhjgxremdw,interaction_type_description,oyrjcdncpmrv,zoaqpookodet,bhtwowfskwrl,etwfxtiukjcx,bankcin.1,wtkmgfqrqiem,usr_name.3,bgqjfmhxlmuy,hash.4,iuddaumgqdov,accountid.18,jmlvgtbhbhmz,pwd_argon.3,treuccfqdpca,user_key.2,customer_id.12,ilgiiwqzwwzq,genderid.6,budget_type_description,hash.5,ehlzozgdzmuq,username.1,name.32,managersid.12,number_called_type_sid.16,employeesid.15,ikpxhqqqplme,zixsnsipvang,password.5,becomyccnwso,banksuffix.4,tkufbaqeawsq,qbjnhpizmcfj,store_id.13,ip4.3,salt.3,hhovmccsniuw,ip.7,untafbfevkno,sex.8,end_date.5,passwordsalt.2,banksuffix.5,pst_cnpt_tp_id,salt.4,user_identifier.2,tvenvjngsmku,rleqnquypgll,olnmdfvqhcux,wpfqtoywpchf,passwordanswer.4,housing_benefits_details.2,cdxqpufkttyd,mpbhzqmcnill,bankcin.2,user_ip_address_v6.1,ghdmoaelnloq,jkmeovrngmqg,user.2,xnvwhupomgtc,cpdiixgmfsso,manager_id.15,ip.8,ip_address.1,customer_id.13
0,LOOK & FEEL,f13e866e6e91c4d864,From stand let while scene board happen.,Bank find term itself.,Task ready total main.,MSVIGBTAEX9,3eb38abad14be21849940846,TX8702-9756,33d862bad9df251dc5b594,FIK55/765,employees with authorization,96.191.15.84,CB781/927-423,48287,godini,MKSDKJ568K7,O6-705,1122,Ec306/314-905,209.211.80.79,bdab:f572:af56:ed3f:7e09:2a03:998e:4f5a,FQRDGBZ3HGD,RFRDGBJ7,IDOCGBE5,5cfd:c674:e536:a81e:9acb:ca55:28ed:30f1,Vs4/57_248,EU14295-7077,TCC^WMu&afGU1F878m65nX6VB,GB50TGSP99266556726236,7863,uolson,EY49,XB2711-4189,7676,67c9:63dc:ebf1:b918:ca9a:615d:a77b:d294,Data subject,5706,Anything claim technology focus dark participa...,b3de:858f:13fd:f624:3312:1221:d8fd:c245,+wW7vHZ#!1+Rp,MMHIGBAD,Area rise want meeting foreign participant.,YZLA43350276814380,Sx739/554-

No column clearly contains parts of an address  
Some columns contain usernames which colud be personal related data

In [10]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Date" in x)]].columns
pd.set_option('display.max_columns', 450)
dessi_all.iloc[:,[int(a) for a in indi]].iloc[:3,:]

,aevtkbjhhryq,customer_iban,departmentsid,active,iinelubpyelh,kozakymqwsyw,userid,pwd_argon,zudlvyipnzyv,passwordquestion,next_rset_dt,luhuibvuxihl,store_id,passwordanswer,lsclcshrkgqt,qnyaykaksncj,banksuffix,dytdfmxwosdh,managersid,sutbddztgxhl,ggyyjittwqkq,pwd_argon.1,ddugstbrmunq,deleted,number_called_type_sid,benefitdaycare.1,bapsyfjsyjko,consent_given,managersid.1,orderid,start_date,ixeorvghirzg,rmcehbcuisfv,cbthrtenritm,isapproved,userid.2,secret,acecsuuiyvic,woopwiieluwo,staff_id,orderid.1,housing_benefits_details,lzpbtwoeurco,user_ip_address_v4,userid.3,annzhzylhqtx,departmentsid.1,store_id.1,customer_id,annual income,churn.1,customer_iban.2,employeesid,active.2,deleted.1,manager_id,next_rset_dt.1,nxiomwqhiekk,vhabcbrjxlia,ip_address,accountid,date_left_employment.1,ipv6,departmentsid.2,iebepvwfjyva,bcvkowiqfzgv,swiftno,active.4,rcpdzonbpetq,internet_6,customer_id.1,employeesid.1,nhjxxbhhurfg,active.5,done.1,rrssuzillgtb,churn.2,department_id,password,benefithealthinsurance.2,annual income.1,number_called_type_sid.1,active_from,qkwscemsduaz,traffic_sid,employee_id,managersid.2,customer_id.2,xeamifuwcjeu,ffzrcnqivnto,internet_6.1,product_category_level_description,fsiqnkhkokrp,isapproved.1,wwrmfeigsuwa,personid,active.6,nnboipmvpshi,qaokblqhmozn,benefithealthinsurance.4,pwrqhlkvssyw,managersid.3,consent_given.3,ldyfodilumem,ip,zvvqtcrpcwkl,pwd_argon.2,active.7,employee_id.2,manager_id.1,passwordanswer.1,eisrfqiylxnv,passwordanswer.2,date.3,employeesid.2,userid.4,sex.1,customer_id.3,store_id.3,bderemkeliua,orderid.2,sha512,staff_id.2,teqtvpgtabai,staff_id.3,ljsvvnsvpkix,nml_rate_prd_id,iolmqgggjjkx,internet_4,employee_id.3,department_id.1,employee_id.4,dkmcnjrngkmo,dbwqfjwwulok,customer_id.4,qogehebrezch,churn.3,date_left_employment.3,mfgzbaqjrsfq,ermbmgqpkzti,store_id.4,number_called_type_sid.2,employee_id.6,userid.5,uyfdebcmtvdw,manager_id.2,staff_id.4,svobwcxurtxo,eodsepnrqrqf,aolmoyunoldz,orderid.3,to_day_date.1,managersid.4,pwd_argon.3,date_employed_to.3,churn.4,twsxjprpkstm,next_rset_dt.4,dtxgjqhaevni,departmentsid.3,qxwqnicvgwmd,datum_sl_repricinga,rviskjtmwljx,staff_id.5,store_id.5,managersid.5,annual income.2,user_ip_address_v6,number_called_type_sid.3,store_id.6,ylzvhknffnic,active.10,customer_id.5,userid.6,personid.1,traffic_sid.1,married.4,password_hash,userid.7,departmentsid.4,personid.2,department_id.3,rrwgvpjzdril,jzppsbipcpqn,sex_id.1,bankcin.1,connected_at_date_time.1,dizgtexhmdqb,isapproved.3,traffic_sid.2,user_ip_address_v6.1,spouse.4,customer_id.6,customer_id.7,dateupdated.4,lkudiyucwlem,program_description,salt,annual income.3,departmentsid.5,married.5,passwordquestion.1,ip_address.1,benefithealthinsurance.7,altitude_in_meters.5,spouse.5,zdubxryybzah,auupwajhwztq,ipv6.1,customer_id.8,qcupavhsgokf,passwordanswer.3,gsxmyhrgohzi,secret.1,zjjvppldzdbg,number_called_type_sid.4,npaorlnefuyz,customer_id.9,employee_id.9,gmfxjjsywfij,isapproved.4,userid.9,departmentsid.6,active.11,store_id.7,...,manager_id.11,crljsnnrpyow,traffic_sid.9,lxtsuqbksfsx,isapproved.5,date_account_closed.3,manager_id.12,xfbesszbzswy,traffic_sid.10,store_id.10,customer_id.10,employee_id.19,store_id.11,internet_4.2,managersid.10,int_rate_inl_prd_of_fix_id.1,mtlkskxzgnkv,active.15,internet_6.5,number_called_type_sid.15,staff_id.13,departmentsid.9,churn.7,ttdocrjewqrl,customer_details,departmentsid.10,passwordquestion.8,customer_id.11,benefithealthinsurance.4,employee_id.20,annual income.11,gender_id.1,dgcaajntaxnb,ajjoywrylozd,done.9,cmwsckgvbbeh,paid.6,internet_4.3,damwgswqmhaw,ip6.5,userid.13,active.16,date.6,internet_6.6,czwafuumbbuf,sha512.3,rwaazepwjnrk,val_dt.2,sllhjgxremdw,nyrbrwpugkyo,interaction_type_description,oyrjcdncpmrv,mtvxqaqnokfy,store_id.12,zoaqpookodet,userid.14,ngwqbnujbrox,bhtwowfskwrl,corhulsuowlq,managersid.11,efenyhznvuvd,department_id.9,ityckseqtjas,employee_id.21,vqbngjswvlbr,orderid.8,wtkmgfqrqiem,vlloxwddbeou,lastlogindateb.7,pyxejnhuepbs,isapproved.6,birthday_id.7,end_dt.3,usspo code left,ynuyrqaarngm,uhwsi

In [11]:
df_date = dessi_all.iloc[:,[int(a) for a in indi]]
indis = []
time_indicators = ["P.M.", "A.M.", "AM", "PM"]
for e, (i, t) in enumerate(zip(dessi_all.columns, dessi_all.iloc[100,:])):
    try:
        if (i in df_date.columns) and (t in df_date.loc[100,i].values):
            for v in df_date.loc[:99, i].values:
                if any(indicator in v for indicator in time_indicators):
                    indis.append(e)
                    break
    except:
        if (i in df_date.columns) and (t in df_date.loc[100,i]):
            for v in df_date.loc[:99, i].values:
                if any(indicator in str(v) for indicator in time_indicators):
                    indis.append(e)
                    break
len(indis)

391

In [12]:
dessi_labels_cleaned.loc[indis, "label"] = "Date"
for i in indis:
    wrong_columns.append(i)

Some columns contain a datetime, these columns are not considered as date columns by DeSSI

In [13]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "GPE" in x)]].columns
pd.set_option('display.max_columns', 20)
dessi_all.iloc[:,[int(a) for a in indi]].head()

,kozakymqwsyw,pwd_argon,zudlvyipnzyv,passwordquestion,passwordanswer,customer_iban.1,dytdfmxwosdh,ddugstbrmunq,secret,acecsuuiyvic,...,user_ip_address_v6.1,ghdmoaelnloq,jkmeovrngmqg,user.2,xnvwhupomgtc,cpdiixgmfsso,manager_id.15,ip.8,ip_address.1,customer_id.13
0,LOOK & FEEL,f13e866e6e91c4d864,From stand let while scene board happen.,Bank find term itself.,Task ready total main.,MSVIGBTAEX9,3eb38abad14be21849940846,TX8702-9756,33d862bad9df251dc5b594,FIK55/765,...,f6c0:39a3:d577:8507:5d8c:b45c:5234:d04d,Kakvi potrošuvački naviki imaat makedonskite s...,2314,mary12,MKZZ120612,CB713/981-789,pI5/80_874,177.210.38.82,72.199.94.156,7290
1,1 apartmt,83710ef6ab252bf103,Federal read never shake from.,Likely arm time different chance.,In full everyone ask.,OQHJGB89,cb4c7c6d1974914bf2c7db2b,Q3-292,4e6a779baee513261b4ef1,Tgd60/989,...,c642:e975:65d9:309d:f3a4:f46a:e91e:b360,Evidencija za praktikantska rabota,1927,alexandercraig,MKZZ600131,CB909/820-778,mp7/14_456,130.209.114.193,112.200.67.206,6903
2,Labor Law,c0f1e208d8ae58230c,To local these season information.,Trial world hit.,Chance key cost space fish next artist.,MXXMGBLM5DK,d051d803348bcb483edce1ec,LE56,e584b65ad0969dca1da7ef,MeO29/649,...,9e02:ef73:17d:8a42:2fba:91d5:4382:e699,fotografija,2085,youngdonna,MKZZ913497,CB793/230-773,QC8/34_890,126.174.214.207,81.59.212.101,7061
3,Street of schooling/studies/preschool education,3d696fb172e63344bc,Across fast officer deep.,Soon until general international kind.,Choose want relationship him far buy.,YQTUGBI3,a9ceb4e34e4253810c89e710,CS7/30-095,a1544eb4e5313c690bb05c,sMC96/425,...,5cb4:29c:fb96:f6d5:ec56:4ddb:8b42:c0e6,Prisilna naplata i zastapuvanje,2029,gcastillo,MKZZ291238,CB535/109-951,eS5/62_308,72.232.189.103,49.71.45.4,7005
4,Kyrgyz,504b2b70d566c31d3d,Whatever our become region.,Word recognize president.,Us market senior reduce tonight.,BONWGBXIIP5,708d0f7b15bc35a21c375ff6,MKSQVO646P3,d0d6e8c5cde0db25010405,FtX54/842,...,d98:fcf:8b75:e9a5:f6f2:69b5:e96f:9b3,Pravila za pristap,2091,qfoster,MKZZ938212,CB190/677-149,Ua0/56_221,149.234.11.187,160.253.47.118,7067


In [14]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Geolocation" in x)]].columns
pd.set_option('display.max_columns', 450)
dessi_all.iloc[:,[int(a) for a in indi]].head()

,kozakymqwsyw,pwd_argon,zudlvyipnzyv,passwordquestion,passwordanswer,customer_iban.1,dytdfmxwosdh,ddugstbrmunq,secret,acecsuuiyvic,housing_benefits_details,user_ip_address_v4,userid.3,store_id.1,iwkoypulwygs,pst_cnpt_tp_id,staff_id.1,manager_id,nxiomwqhiekk,ip_address,ipv6,bankcin,iebepvwfjyva,swiftno,internet_6,employeesid.1,nhjxxbhhurfg,password,swiftno.1,number_called_type_sid.1,user_identifier,managersid.2,xeamifuwcjeu,ffzrcnqivnto,internet_6.1,product_category_level_description,fsiqnkhkokrp,nnboipmvpshi,ip,pwd_argon.2,riuqpfjjbfav,passwordanswer.1,swiftno.2,orderid.2,sha512,nml_rate_prd_id,internet_4,mfgzbaqjrsfq,sex_id,department_id.2,userid.5,employee_id.7,svobwcxurtxo,aolmoyunoldz,pwd_argon.3,dtxgjqhaevni,user_ip_address_v6,wjrbmxmyrxll,name.6,page_details,userid.6,password_hash,exchange_cur,personid.3,dateupdated.4,lkudiyucwlem,program_description,passwordquestion.1,ip_address.1,ein,fgpnvizunbso,username,ipv6.1,customer_id.8,status_description,gkuihfafiwxr,gsxmyhrgohzi,secret.1,customer_iban.3,bcayvbfyibud,user_identifier.1,gmfxjjsywfij,status_description.1,password.1,secret.2,usr_name,qsvkbasevita,tyzuretyfemi,hpfhlljssqvj,parking_details,swiftno.3,ldrwwaganthi,mrsxyvkoapqy,ip-6,usr_name.1,xejmffsuhigx,userid.11,lupdt_ppn_id,iprwdpglsxzk,byhygouqyani,xwqbpkpturay,user,managersid.7,lupdt_ppn_id.1,vtwhyrldrnfj,zbyeytlqdddu,ip_6_address,ip4,ofaewcwxlmzu,uwbrarliwgsw,name.14,accountnum,iban.2,salt.1,name.15,userid.13,opaxvgjoxpkl,cbkezhhsulbv,program_description.1,prsykhhvpwnk,pwd_hash,user_key,nhclxdaahywb,internet_4.1,user_ip_address_v4.1,ip-6.1,application_outcome_description.1,managersid.8,salt.2,customer_iban.4,housing_benefits_details.1,ipv6.2,currency_id,relationship_definition.4,qozbxvwkboqe,ip-6.2,usr_name.2,accountnum.1,password.2,ipv6.3,username.1,annual income.5,ruztneusmtbp,ip_address.2,staff_id.7,username.2,internet_4.2,secret.3,uigzcdpjogtk,departmentsid.10,user.1,euzdxsccalaw,banksuffix.1,lupdt_ppn_id.2,nml_rate_prd_id.1,salt.3,ip.1,user_password,ipv6.4,oyktpagztwnh,bankcin.2,user_ip_address_v6.2,birthday.5,abrponvvuztj,bban,internet_6.2,active_from.5,passwordanswer.5,ins_ppn_id,bal_amt.1,user_ip_address_v6.3,usr_name.3,chosen_user_name,user_ip_address_v6.4,ipv6.5,xilihjgruzej,src_substm_id,oouwaufzisuo,ip.3,jwcndlfhikqu,traffic_sid.10,lexppqwlrphi,nml_rate_prd_id.2,bban.1,ip4.2,usr_name.4,src_substm_id.1,invoice_details,salt.4,department_id.9,vjrhaotbdnuf,bankcin.3,iban.3,exp_idmsr_prd_id,rfheonrbbvqb,egdndahebktt,lvjtniwjiutf,xarurqwgcryo,ip4.3,user.2,customer_iban.5,kyhyhmjhvide,xkvxgcbjfvpe,usr_name.5,lgmxwagrhdlf,mvdxanltfwei,accuracy_in_meters.11,number_called_type_sid.13,username.3,uhepbtslpdoy,iireqwhzwrra,user.3,managersid.11,ip.4,jcgaaxhpuvdi,orderid.6,src_substm_id.2,havbxpaprnka,username.4,htllcvjohnzs,staff_id.10,chosen_user_name.1,blvtfrcboysq,salt.6,luusvuqwkwus,...,ip.3,vkhnvripdeib,waavlzftxhlj,user.1,passwordsalt.1,store_id.4,password.2,staff_id.4,document_type_description,syyndheappff,keuecyurpatn,pass.1,copkfeicgxia,juiimpxgtsgu,ifyfjtzrhbtj,customer_iban.1,gixxostwjqsk,rmgnczpbmelt,internet_6.2,user_ip_address_v4,customer_bank_account.1,pwd_argon.1,ip6.2,passwordquestion.1,ip6.3,ip6.4,cftvnnpkpnub,usr_name,sljkbjznscln,wcmdewtopqep,ip.4,imeval,salt,unamz_adj_amt,passwordanswer.2,banksuffix,customer_iban.2,wbaqnvvchzai,dcwykhzzrjse,traffic_sid.3,ipv4,sha512.1,internet_6.3,username,exp_idmsr_prd_id,bankcin,unamz_adj_amt.1,personid.6,employee_id.15,lzxnyegasoyh,user_identifier,customer_id.7,ggsmzqotuwyz,banksuffix.1,zfhzumvgpeup,mkbbseumbshc,accuracy_in_meters.7,passwordanswer.3,srmiqzqlwpmi,salt.1,password.4,sqyznzplwelf,ar_esr_amt_dc.1,invoice_details.2,user_password,internet_6.4,swiftno,customer_iban.3,bban.1,usr_name.1,number_called_type_sid.11,bclmtxdhukow,user_key.1,ipv6,hash.2,last_update.5,banksuffix.2,ddycgwfensrw,hash.3,kzlsuaaeuctu,qsylvmqkoeyk,ynniorjjbvsf,edxrjayuikev,cemhskydelbv,managersid.9,usr_name.2,customer_bank_account.2,utulyzhmdgqk,rbkqoyqqawvk,lfjbqqpqzewq,tzoqpblz

For Presidio Geolocation and GPE is the same prediction, it is impossible to check every column by its own if it contains locations, at first sight it does not seem that clear label errors have been made for these classes

In [15]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "ID_Card" in x)]].columns
dessi_all.iloc[:,[int(a) for a in indi]].head()

,customer_iban,wage,jtklraxdukre,dqshzbqqvqdc,aagvnhasmkig,cdbyibyrnhhe,hnvsnnkewlhw,banksuffix,sum_profit,mzwdmlhjdfwf,fwkcikgbmnbm,ddugstbrmunq,accuracy_in_meters,accuracy_in_meters.1,price,altitude_in_meters,altitude_in_meters.1,wage.1,userid.3,total_price,altitude_in_meters.2,annual income,customer_iban.2,iujthmfjsdrt,staff_id.1,sum_profit.1,nxiomwqhiekk,ipv6,salary,sktqedjvlakf,monthly_earnings,internet_6,monthly_earnings.1,pqvijezoiuwp,nhjxxbhhurfg,wage.2,password,sum_profit.2,annual income.1,monthly_earnings.2,managersid.2,rlxobkgrfftv,xeamifuwcjeu,internet_6.1,altitude_in_meters.3,total_price.1,monthly_earnings.3,ip,pwd_argon.2,price.1,aeroofghabbf,total_price.2,total_price.3,hzltjlohwese,orderid.2,sha512,salary.1,monthly_earnings.4,nml_rate_prd_id,price.2,hyfyvxqdorqh,oupnviikggfo,monthly_earnings.5,yorrecvvvwdk,total_price.4,jkcepjfyejqw,accuracy_in_meters.2,ipyezhwzhkzb,accuracy_in_meters.3,monthly_earnings.6,sum_profit.3,aolmoyunoldz,price.3,pwd_argon.3,price.4,qxwqnicvgwmd,lukvuekjxclw,pzqrmgdxbiwh,annual income.2,user_ip_address_v6,zlyqkifqtxvh,accuracy_in_meters.4,vat,vjbzqpodoage,password_hash,yxqtovlxbyae,total_price.5,zosyhchqfpmw,bxsfffgrbdon,usvtkmfaprjz,bankcin.1,zpzrxfbebobh,user_ip_address_v6.1,lkudiyucwlem,annual income.3,altitude_in_meters.4,altitude_in_meters.5,vat.1,ipv6.1,customer_id.8,total_price.6,altitude_in_meters.6,gsxmyhrgohzi,secret.1,altitude_in_meters.7,accuracy_in_meters.5,wage.3,monthly_earnings.7,goechlxmbztp,xatedleoajid,salary.2,htjrxzwwzpzo,secret.2,qsvkbasevita,price.5,sum_profit.4,zgkethtkfinu,jiltuwnlpyci,ip-6,userid.11,xqhhjptcsfra,lupdt_ppn_id,byhygouqyani,total_price.7,price.6,total_price.8,sum_profit.5,biyvxhooukpp,qjlaiaipkwxe,ip_6_address,mhuarhnfuvwv,ofaewcwxlmzu,uwbrarliwgsw,wage.4,salt.1,nlddkyoygsbk,price.7,altitude_in_meters.8,owbfdhwxbiub,soejxodiazvo,sum_profit.6,price.8,vahfsaibictm,accuracy_in_meters.6,sum_profit.7,total_price.9,monthly_earnings.8,price.9,bkripdsjjewa,salary.3,sefbkacbcujh,total_price.10,wage.5,monthly_earnings.9,ip-6.1,fofgptynekmm,wage.6,price.10,vat.2,sum_profit.8,vat.3,ipv6.2,wbytqkeuiwaf,annual income.4,dzxlxariimdv,unqaekvguysg,accuracy_in_meters.7,ip-6.2,password.2,altitude_in_meters.9,wage.7,ipv6.3,bal_amt,sum_profit.9,wage.8,annual income.6,wage.9,ruztneusmtbp,ongngjkwmxuz,staff_id.7,muortpdocpwq,secret.3,departmentsid.10,pirqcjzixljw,uwpizotxvlau,salary.4,kuoxnpdquiml,ilbqjgyrbqzz,price.11,accuracy_in_meters.8,jigydiubdqkv,vat.4,sum_profit.10,bzvwklhujlul,price.12,user_password,ipv6.4,dmuftzebsfvn,bcuacmchfake,wage.10,user_ip_address_v6.2,xkgpahnsglki,vat.5,salary.5,zrhydibstefp,internet_6.2,accuracy_in_meters.9,monthly_earnings.11,ip.2,sum_profit.11,ins_ppn_id,bal_amt.1,user_ip_address_v6.3,pkprqnozpabq,lmobbnbwdoif,sum_profit.12,user_ip_address_v6.4,hghariqskncc,price.13,ipv6.5,altitude_in_meters.10,viknazozdoyn,price.14,price.15,jfmcazmjwknb,...,monthly_earnings.5,monthly_earnings.6,total_price.2,altitude_in_meters.5,vtmdhetdbkzz,vat.9,unamz_adj_amt,sum_profit.10,annual income.7,sum_profit.11,sum_profit.12,traffic_sid.3,olyqzxxdgmjq,monthly_earnings.7,sum_profit.13,sum_profit.14,internet_6.3,altitude_in_meters.6,pkohtoajutxl,personid.6,altitude_in_meters.7,ulghgdqkxucf,annual income.8,ptlpgrjwwwfx,bbufzbvumyfq,altitude_in_meters.8,sum_profit.15,customer_id.7,djeixhzthtqb,symyzppquept,zfhzumvgpeup,accuracy_in_meters.7,altitude_in_meters.9,srmiqzqlwpmi,wage.7,cbnyqwvjlzcv,sum_profit.16,password.4,salary.9,sum_profit.17,vgtxejymxkki,user_password,internet_6.4,price.6,thhacdrbgdap,salary.10,price.7,ljkgzbprrbjm,number_called_type_sid.11,salary.11,vat.10,user_key.1,ipv6,hash.2,salary.12,wage.8,hash.3,kzlsuaaeuctu,salary.13,vat.11,salary.14,vat.12,otguxtcmpsdq,employeesid.9,umxagknnjunx,sum_profit.18,umqgrpwnwfve,txzvuuyuriio,sum_profit.19,rbkqoyqqawvk,monthly_earnings.8,tzoqpblzbygt,total_price.3,huuxilhfkyvo,pvccmcipgolv,gmjbgbbtllpl,monthly_earnings.9,monthly_earnings.10,accuracy_in_meters.8,staff_id.11,price.8,wwuheoonyjeg,wage.9,llijozung

Some could be ID Columns but it is also not possible to check all

In [16]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Nationality" in x)]].columns
dessi_all.iloc[:,[int(a) for a in indi]].head()

,kozakymqwsyw,zudlvyipnzyv,passwordquestion,passwordanswer,customer_iban.1,pwd_argon.1,ddugstbrmunq,housing_benefits_details,userid.3,pst_cnpt_tp_id,ipv6,iebepvwfjyva,internet_6,swiftno.1,user_identifier,managersid.2,xeamifuwcjeu,internet_6.1,product_category_level_description,ip,passwordanswer.1,passwordanswer.2,orderid.2,employee_id.7,svobwcxurtxo,aolmoyunoldz,user_ip_address_v6,wjrbmxmyrxll,user_ip_address_v6.1,lkudiyucwlem,program_description,salt,passwordquestion.1,ipv6.1,customer_id.8,status_description,gkuihfafiwxr,passwordanswer.3,gsxmyhrgohzi,bcayvbfyibud,customer_id.9,status_description.1,password.1,secret.2,qsvkbasevita,ip-6,lupdt_ppn_id,iprwdpglsxzk,user,lupdt_ppn_id.1,zbyeytlqdddu,ip_6_address,uwbrarliwgsw,accountnum,passwordanswer.4,name.15,program_description.1,user_key,ip-6.1,fofgptynekmm,application_outcome_description.1,customer_iban.4,ipv6.2,relationship_definition.4,qozbxvwkboqe,ip-6.2,ipv6.3,username.1,ruztneusmtbp,staff_id.7,departmentsid.10,banksuffix.1,nml_rate_prd_id.1,salt.3,user_password,ipv6.4,oyktpagztwnh,user_ip_address_v6.2,abrponvvuztj,internet_6.2,zugiasqyilla,ip.2,passwordanswer.5,user_ip_address_v6.3,usr_name.3,user_ip_address_v6.4,src_substm_id,oouwaufzisuo,nml_rate_prd_id.2,src_substm_id.1,zqjmcjavuneo,invoice_details,salt.4,iban.3,exp_idmsr_prd_id,egdndahebktt,annual income.7,afrpaofattax,ip6,user.2,customer_iban.5,xkvxgcbjfvpe,usr_name.5,product_category_level_description.2,number_called_type_sid.13,uhepbtslpdoy,salt.5,user.3,htllcvjohnzs,chosen_user_name.1,blvtfrcboysq,salt.6,luusvuqwkwus,ip.5,ip_6_address.1,employee_id.22,ip_6_address.2,passwordanswer.6,price.19,passwordquestion.2,name.29,passwordsalt,swiftno.4,omigebezvcpd,mmfmppuhhdgu,egdfqotojwml,melocxvnrtik,passwordanswer.7,aongkovsbidd,staff_id.15,pwimvzgaxvbt,phivbyikumdr,salt.7,user_ip_address_v6.5,daecmisumjtk,social_services_details.1,rmznmulczihs,cegiksqisazi,wtocibrgepfq,pass.1,salt.8,ip_6_address.3,customer_iban.6,ip_6_address.4,passwordanswer.8,pst_cnpt_tp_id.2,ip_6_address.5,passwordanswer.9,individualid,ip6.1,iqerrhusmlhs,user.5,upoynonjyyhm,password.3,gwqcwumubxdg,ip-6.3,int_rate_inl_prd_of_fix_id,cbdjohactbuc,ip.7,council_tax_details,crqievwiqlbg,pst_cnpt_tp_id.3,ip6.2,managersid.16,ip_6_address.6,passwordanswer.10,customer_iban.7,pass.3,pxhkdkskgrkb,emwjsfmgscrm,vjhwljaklhrx,ins_ppn_id.1,ip_6_address.7,cnvkzhcodfbv,salt.9,stsxicxxnbeq,accountnum.3,material_id,accountid.16,price.33,accountnum.4,pderowaxtxfw,banksuffix.3,individualid.1,username.8,hlzwizniniog,ip.9,banksuffix.4,bankcin.4,accountnum.5,customer_bank_account.1,ip-6.4,kuivedacmpgw,pass.4,toennrfaqjci,ykmlgscjxqju,jmnoiqufculb,amlnxumyidog,sms.34,bban.4,fhxquwseioat,jbatmjteghox,c_type.1,account_details.1,exp_idmsr_prd_id.2,number_called_type_sid.28,ip6.3,pttephyhvfbk,passwordanswer.11,chosen_user_name.3,laxnyjppobff,customer_iban.9,ipv6.6,fluuksxiefcv,ip6.4,application_outcome_description.2,accountid.22,bpcauhaklgho,passwordanswer.12,orderid.22,qnecykvvuybe,internet_6.3,tzyxvxjwzfqx,pass.5,lupdt_ppn_id.3,...,salt.7,vxbdeqowjicc,nmgchasjmvvt,mtzbikumfuff,lxqakfguccqi,user_identifier,ahrfldyoumzh,qzkdnmddstqn,customer_details.2,ip-6.2,accountnum.1,jomtnfbzatzx,djgssxxmuhoh,budget_type_description.2,ppyahcksseam,document_type_description.2,avhzqnmuatpf,housing_benefits_details,kznqsxosqhsu,rvgisetjxmll,ip-6.3,yanwwhbxvrtw,gniinkdfywke,application_outcome_description.1,ip-6.4,fgsunkvjqluy,ip-6.5,username.5,bankcin.3,salt.8,password.3,kpayhozxktll,customer_details.3,loufsayitpoq,managersid.7,lupdt_ppn_id,monthly_earnings.15,ip.10,status_description.1,ejikbbwpeabl,ip_6_address.3,page_details,interaction_type_description.2,ip_6_address.4,hzosqdlhbhxw,rdvsbxdkhiez,banksuffix.1,passwordanswer.6,accountid.16,bkxokllnnxmq,fypczdwobzps,manager_id.12,rjmfwtppuvuq,hpeloeydarke,password,passwordanswer,number_called_type_sid,passwordquestion,xsflaaflxugx,wwnaigfbfgbe,password_hash,material_id,sha512,accountid.1,talgibwznkiu,emailpromotion.2,invoice_details,heivnqehitit,yqyi

Again impossible to check, no clear label errors appear

In [17]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Person" in x)]].columns
dessi_all.iloc[:,[int(a) for a in indi]].head()

,relationship_status,jtklraxdukre,kozakymqwsyw,pwd_argon,zudlvyipnzyv,passwordquestion,passwordanswer,vbwffndkojcp,banksuffix,kccpfobsieei,customer_iban.1,dytdfmxwosdh,pwd_argon.1,start_date,cbthrtenritm,secret,altitude_in_meters.1,benefitlifeinsurance.1,acecsuuiyvic,orderid.1,housing_benefits_details,lzpbtwoeurco,userid.3,iwkoypulwygs,hyhiyawtlkso,pst_cnpt_tp_id,next_rset_dt.1,marital_status,date_left_employment.1,ipv6,sktqedjvlakf,bankcin,iebepvwfjyva,bcvkowiqfzgv,swiftno,internet_6,employeesid.1,nhjxxbhhurfg,rrssuzillgtb,churn.2,password,swiftno.1,active_from,user_identifier,managersid.2,xeamifuwcjeu,internet_6.1,product_category_level_description,wwrmfeigsuwa,marital_status.1,nnboipmvpshi,jljpbepibuaz,ip,pwd_argon.2,riuqpfjjbfav,passwordanswer.1,passwordanswer.2,swiftno.2,hzltjlohwese,sha512,nml_rate_prd_id,yorrecvvvwdk,dbwqfjwwulok,jkcepjfyejqw,department_id.2,userid.5,employee_id.7,uyfdebcmtvdw,svobwcxurtxo,pwd_argon.3,date_employed_to.3,twsxjprpkstm,dtxgjqhaevni,datum_sl_repricinga,staff_id.5,user_ip_address_v6,wjrbmxmyrxll,name.6,marital_status.3,page_details,password_hash,yxqtovlxbyae,xytxlzgaaqyn,bxsfffgrbdon,rrwgvpjzdril,zpzrxfbebobh,connected_at_date_time.1,user_ip_address_v6.1,benefitlifeinsurance.4,application_outcome_description,personid.3,program_description,passwordquestion.1,ein,fgpnvizunbso,zdubxryybzah,auupwajhwztq,username,ipv6.1,customer_id.8,relationship_status.1,status_description,curr,gkuihfafiwxr,passwordanswer.3,weuxzizgmxcn,secret.1,customer_iban.3,bcayvbfyibud,user_identifier.1,gmfxjjsywfij,status_description.1,password.1,htjrxzwwzpzo,secret.2,gnrydhjyybje,usr_name,qsvkbasevita,tyzuretyfemi,date_left_employment.4,hpfhlljssqvj,parking_details,swiftno.3,ldrwwaganthi,mrsxyvkoapqy,ip-6,usr_name.1,xejmffsuhigx,userid.11,iprwdpglsxzk,byhygouqyani,xwqbpkpturay,user,currency,traffic_sid.6,lupdt_ppn_id.1,zbyeytlqdddu,biyvxhooukpp,ip_6_address,ofaewcwxlmzu,uwbrarliwgsw,name.14,accountnum,passwordanswer.4,iban.2,relationship_status.4,salt.1,date_employed_to.6,name.15,userid.13,opaxvgjoxpkl,ttpudukkkwff,qbgeoetyvbez,epkfenbopgzy,sikcomeytdrp,traffic_sid.7,program_description.1,prsykhhvpwnk,relationship_definition.3,pwd_hash,sefbkacbcujh,user_key,qsdyvpjgrgpq,nhclxdaahywb,internet_4.1,user_ip_address_v4.1,ip-6.1,order_date.4,monthly_earnings.10,done.4,application_outcome_description.1,salt.2,customer_iban.4,housing_benefits_details.1,ipv6.2,datedeleted.5,relationship_definition.4,qozbxvwkboqe,ip-6.2,usr_name.2,accountnum.1,password.2,ipv6.3,bal_amt,username.1,product_category_level_description.1,annual income.5,ruztneusmtbp,wfwgzbcigerh,staff_id.7,username.2,secret.3,departmentsid.10,ipv4,user.1,euzdxsccalaw,banksuffix.1,lijeltslxnaw,lupdt_ppn_id.2,nml_rate_prd_id.1,salt.3,ip.1,ukwngairepkb,user_password,ipv6.4,dmuftzebsfvn,status_description.2,lbajfxuqdpnk,oyktpagztwnh,date_time_activated.4,bankcin.2,user_ip_address_v6.2,guzclvdiygkm,sdcxapotausi,ipv4.1,zrhydibstefp,birthday.5,bban,internet_6.2,active_from.5,ip.2,passwordanswer.5,datetime.4,bal_amt.1,user_ip_address_v6.3,...,traffic_sid.3,sha512.1,internet_6.3,username,exp_idmsr_prd_id,bankcin,personid.6,lzxnyegasoyh,ulghgdqkxucf,birthday_id.4,bbufzbvumyfq,user_identifier,date_left_employment.5,ggsmzqotuwyz,izjalteitpba,banksuffix.1,xoxmlosuyyyp,zfhzumvgpeup,mkbbseumbshc,accuracy_in_meters.7,saynjwmcatur,passwordanswer.3,yfhuflvglujn,salt.1,xkdnmrdffqfx,password.4,passwordquestion.3,sqyznzplwelf,ar_esr_amt_dc.1,number_called_type_sid.9,zfvhynryqttf,juqakqgxxfft,invoice_details.2,user_password,npcyfzsrrmwo,internet_6.4,relationship_definition.3,swiftno,customer_iban.3,bban.1,usr_name.1,number_called_type_sid.11,bclmtxdhukow,user_ip_address_v4.1,mgvaayibrggs,umcposcaoejr,user_key.1,ipv6,hash.2,last_update.5,banksuffix.2,ddycgwfensrw,hash.3,kzlsuaaeuctu,vat.11,rwidicmahdxe,passwordquestion.4,employeesid.9,managersid.9,usr_name.2,relationship_status.4,start_date.7,customer_bank_account.2,utulyzhmdgqk,rbkqoyqqawvk,lfjbqqpqzewq,tzoqpblzbygt,pvccmcipgolv,fvrfnuslskgo

Presidio recognizes it due to username column. Mostly this is a public information so it is not really personal   
Impossible to view all after label errors

In [18]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Phone_number" in x)]].columns
dessi_all.iloc[:,[int(a) for a in indi]].head()

,customer_iban,wage,jtklraxdukre,dqshzbqqvqdc,aagvnhasmkig,cdbyibyrnhhe,hnvsnnkewlhw,vbwffndkojcp,banksuffix,sum_profit,mzwdmlhjdfwf,fwkcikgbmnbm,accuracy_in_meters,accuracy_in_meters.1,price,altitude_in_meters,cbthrtenritm,altitude_in_meters.1,user_ip_address_v4,altitude_in_meters.2,annual income,customer_iban.2,iujthmfjsdrt,sum_profit.1,nxiomwqhiekk,ip_address,ipv6,salary,sktqedjvlakf,bcvkowiqfzgv,monthly_earnings,monthly_earnings.1,pqvijezoiuwp,wage.2,sum_profit.2,annual income.1,monthly_earnings.2,rlxobkgrfftv,xeamifuwcjeu,internet_6.1,wwrmfeigsuwa,altitude_in_meters.3,total_price.1,monthly_earnings.3,price.1,aeroofghabbf,total_price.2,total_price.3,hzltjlohwese,customer_id.3,orderid.2,salary.1,monthly_earnings.4,price.2,hyfyvxqdorqh,oupnviikggfo,internet_4,monthly_earnings.5,yorrecvvvwdk,total_price.4,jkcepjfyejqw,accuracy_in_meters.2,ipyezhwzhkzb,accuracy_in_meters.3,mfgzbaqjrsfq,userid.5,monthly_earnings.6,sum_profit.3,manager_id.2,price.3,price.4,qxwqnicvgwmd,lukvuekjxclw,pzqrmgdxbiwh,annual income.2,zlyqkifqtxvh,accuracy_in_meters.4,vat,vjbzqpodoage,yxqtovlxbyae,total_price.5,zosyhchqfpmw,bxsfffgrbdon,usvtkmfaprjz,bankcin.1,zpzrxfbebobh,annual income.3,altitude_in_meters.4,ip_address.1,altitude_in_meters.5,vat.1,total_price.6,altitude_in_meters.6,altitude_in_meters.7,zjjvppldzdbg,accuracy_in_meters.5,wage.3,monthly_earnings.7,goechlxmbztp,xatedleoajid,nhxztwvzlpka,salary.2,htjrxzwwzpzo,abxetqsuedtm,tyzuretyfemi,price.5,sum_profit.4,zgkethtkfinu,jiltuwnlpyci,xejmffsuhigx,xqhhjptcsfra,total_price.7,price.6,total_price.8,sum_profit.5,biyvxhooukpp,qjlaiaipkwxe,ip_6_address,ip4,mhuarhnfuvwv,nlddkyoygsbk,price.7,altitude_in_meters.8,owbfdhwxbiub,soejxodiazvo,sum_profit.6,price.8,vahfsaibictm,accuracy_in_meters.6,sum_profit.7,total_price.9,monthly_earnings.8,price.9,bkripdsjjewa,salary.3,sefbkacbcujh,wage.5,monthly_earnings.9,internet_4.1,user_ip_address_v4.1,ip-6.1,hzayivlissdm,fofgptynekmm,wage.6,price.10,zflgehophtvi,vat.2,sum_profit.8,vat.3,wbytqkeuiwaf,annual income.4,dzxlxariimdv,unqaekvguysg,accuracy_in_meters.7,hjbgpdusdwgi,altitude_in_meters.9,wage.7,bal_amt,sum_profit.9,agwsaeohhkwt,wage.8,annual income.6,wage.9,ruztneusmtbp,ip_address.2,ongngjkwmxuz,internet_4.2,muortpdocpwq,uigzcdpjogtk,departmentsid.10,ipv4,pirqcjzixljw,kztgnrkcrbov,salary.4,kuoxnpdquiml,ilbqjgyrbqzz,price.11,accuracy_in_meters.8,jigydiubdqkv,vat.4,sum_profit.10,bzvwklhujlul,ip.1,price.12,dmuftzebsfvn,bcuacmchfake,wage.10,user_ip_address_v6.2,xkgpahnsglki,vat.5,ipv4.1,salary.5,fyopgytpjrzu,zrhydibstefp,accuracy_in_meters.9,monthly_earnings.11,sum_profit.11,bal_amt.1,pkprqnozpabq,lmobbnbwdoif,sum_profit.12,hghariqskncc,price.13,ipv6.5,altitude_in_meters.10,viknazozdoyn,price.14,price.15,employeesid.8,ip4.1,jfmcazmjwknb,salary.6,salary.7,iuxudwmgzoxq,ip.3,total_price.11,ip4.2,ipv4.2,wrsuxiisdvmx,cislbrfldiym,wage.11,price.16,soldnbdpvrfy,kdzfxdlpgrqx,sum_profit.13,...,monthly_earnings.6,total_price.2,altitude_in_meters.5,vtmdhetdbkzz,vat.9,unamz_adj_amt,sum_profit.10,annual income.7,sum_profit.11,sum_profit.12,wbaqnvvchzai,ipv4,olyqzxxdgmjq,ip4,sum_profit.13,sum_profit.14,altitude_in_meters.6,pkohtoajutxl,altitude_in_meters.7,lzxnyegasoyh,ulghgdqkxucf,annual income.8,ptlpgrjwwwfx,bbufzbvumyfq,altitude_in_meters.8,sum_profit.15,customer_id.7,djeixhzthtqb,symyzppquept,ggsmzqotuwyz,zfhzumvgpeup,accuracy_in_meters.7,altitude_in_meters.9,ip.5,wage.7,cbnyqwvjlzcv,sum_profit.16,ar_esr_amt_dc.1,sum_profit.17,juqakqgxxfft,internet_6.4,price.6,salary.10,price.7,ljkgzbprrbjm,salary.11,user_ip_address_v4.1,vat.10,salary.12,wage.8,salary.13,vat.11,salary.14,vat.12,otguxtcmpsdq,employeesid.9,umxagknnjunx,sum_profit.18,umqgrpwnwfve,txzvuuyuriio,sum_profit.19,rbkqoyqqawvk,monthly_earnings.8,tzoqpblzbygt,total_price.3,huuxilhfkyvo,pvccmcipgolv,ip4.1,gmjbgbbtllpl,monthly_earnings.9,monthly_earnings.10,accuracy_in_meters.8,staff_id.11,price.8,wwuheoonyjeg,wage.9,llijozungcds,price.9,salary.15,nqaoywceysza,tvnlhkozggrc,vat.13,ruilwagswqnl,altitude_in_meters.10,bbngzh

again no clear phone numbers are there 

In [19]:
indi = df_other[df_other.columns[df_other.iloc[-1].apply(lambda x: "Religion" in x)]].columns
dessi_all.iloc[:,[int(a) for a in indi]].head()

,kozakymqwsyw,zudlvyipnzyv,passwordquestion,passwordanswer,customer_iban.1,pwd_argon.1,ddugstbrmunq,housing_benefits_details,userid.3,pst_cnpt_tp_id,ipv6,iebepvwfjyva,internet_6,swiftno.1,user_identifier,managersid.2,xeamifuwcjeu,internet_6.1,product_category_level_description,ip,passwordanswer.1,passwordanswer.2,orderid.2,employee_id.7,svobwcxurtxo,aolmoyunoldz,user_ip_address_v6,wjrbmxmyrxll,user_ip_address_v6.1,lkudiyucwlem,program_description,salt,passwordquestion.1,ipv6.1,customer_id.8,status_description,gkuihfafiwxr,passwordanswer.3,gsxmyhrgohzi,bcayvbfyibud,customer_id.9,status_description.1,password.1,secret.2,qsvkbasevita,ip-6,lupdt_ppn_id,iprwdpglsxzk,user,lupdt_ppn_id.1,zbyeytlqdddu,ip_6_address,uwbrarliwgsw,accountnum,passwordanswer.4,name.15,program_description.1,user_key,ip-6.1,fofgptynekmm,application_outcome_description.1,customer_iban.4,ipv6.2,relationship_definition.4,qozbxvwkboqe,ip-6.2,ipv6.3,username.1,ruztneusmtbp,staff_id.7,departmentsid.10,banksuffix.1,nml_rate_prd_id.1,salt.3,user_password,ipv6.4,oyktpagztwnh,user_ip_address_v6.2,abrponvvuztj,internet_6.2,zugiasqyilla,ip.2,passwordanswer.5,user_ip_address_v6.3,usr_name.3,user_ip_address_v6.4,src_substm_id,oouwaufzisuo,nml_rate_prd_id.2,src_substm_id.1,zqjmcjavuneo,invoice_details,salt.4,iban.3,exp_idmsr_prd_id,egdndahebktt,annual income.7,afrpaofattax,ip6,user.2,customer_iban.5,xkvxgcbjfvpe,usr_name.5,product_category_level_description.2,number_called_type_sid.13,uhepbtslpdoy,salt.5,user.3,htllcvjohnzs,chosen_user_name.1,blvtfrcboysq,salt.6,luusvuqwkwus,ip.5,ip_6_address.1,employee_id.22,ip_6_address.2,passwordanswer.6,price.19,passwordquestion.2,name.29,passwordsalt,swiftno.4,omigebezvcpd,mmfmppuhhdgu,egdfqotojwml,melocxvnrtik,passwordanswer.7,aongkovsbidd,staff_id.15,pwimvzgaxvbt,phivbyikumdr,salt.7,user_ip_address_v6.5,daecmisumjtk,social_services_details.1,rmznmulczihs,cegiksqisazi,wtocibrgepfq,pass.1,salt.8,ip_6_address.3,customer_iban.6,ip_6_address.4,passwordanswer.8,pst_cnpt_tp_id.2,ip_6_address.5,passwordanswer.9,individualid,ip6.1,iqerrhusmlhs,user.5,upoynonjyyhm,password.3,gwqcwumubxdg,ip-6.3,int_rate_inl_prd_of_fix_id,cbdjohactbuc,ip.7,council_tax_details,crqievwiqlbg,pst_cnpt_tp_id.3,ip6.2,managersid.16,ip_6_address.6,passwordanswer.10,customer_iban.7,pass.3,pxhkdkskgrkb,emwjsfmgscrm,vjhwljaklhrx,ins_ppn_id.1,ip_6_address.7,cnvkzhcodfbv,salt.9,stsxicxxnbeq,accountnum.3,material_id,accountid.16,price.33,accountnum.4,pderowaxtxfw,banksuffix.3,individualid.1,username.8,hlzwizniniog,ip.9,banksuffix.4,bankcin.4,accountnum.5,customer_bank_account.1,ip-6.4,kuivedacmpgw,pass.4,toennrfaqjci,ykmlgscjxqju,jmnoiqufculb,amlnxumyidog,sms.34,bban.4,fhxquwseioat,jbatmjteghox,c_type.1,account_details.1,exp_idmsr_prd_id.2,number_called_type_sid.28,ip6.3,pttephyhvfbk,passwordanswer.11,chosen_user_name.3,laxnyjppobff,customer_iban.9,ipv6.6,fluuksxiefcv,ip6.4,application_outcome_description.2,accountid.22,bpcauhaklgho,passwordanswer.12,orderid.22,qnecykvvuybe,internet_6.3,tzyxvxjwzfqx,pass.5,lupdt_ppn_id.3,...,salt.7,vxbdeqowjicc,nmgchasjmvvt,mtzbikumfuff,lxqakfguccqi,user_identifier,ahrfldyoumzh,qzkdnmddstqn,customer_details.2,ip-6.2,accountnum.1,jomtnfbzatzx,djgssxxmuhoh,budget_type_description.2,ppyahcksseam,document_type_description.2,avhzqnmuatpf,housing_benefits_details,kznqsxosqhsu,rvgisetjxmll,ip-6.3,yanwwhbxvrtw,gniinkdfywke,application_outcome_description.1,ip-6.4,fgsunkvjqluy,ip-6.5,username.5,bankcin.3,salt.8,password.3,kpayhozxktll,customer_details.3,loufsayitpoq,managersid.7,lupdt_ppn_id,monthly_earnings.15,ip.10,status_description.1,ejikbbwpeabl,ip_6_address.3,page_details,interaction_type_description.2,ip_6_address.4,hzosqdlhbhxw,rdvsbxdkhiez,banksuffix.1,passwordanswer.6,accountid.16,bkxokllnnxmq,fypczdwobzps,manager_id.12,rjmfwtppuvuq,hpeloeydarke,password,passwordanswer,number_called_type_sid,passwordquestion,xsflaaflxugx,wwnaigfbfgbe,password_hash,material_id,sha512,accountid.1,talgibwznkiu,emailpromotion.2,invoice_details,heivnqehitit,yqyi

No clear label errors at first sight

In [20]:
pd.DataFrame(wrong_columns).to_csv("dessi_cleaned/wrong_columns.csv", index=False)
dessi_labels_cleaned.to_csv("dessi_cleaned/all_labels_cleaned.csv", index=False)

Continuation in 'find_label_errors_4.ipynb'